In [1]:
import pandas as pd
import numpy as np
import gc 
import math
import pickle
import os
from datetime import datetime
from datetime import timedelta

只提取用户-商品特征，体现用户在购买选择时的选择，用于f12指标。
u模型重点在于：用户是否买cate8商品。us模型重点在于：已知用户要买的前提下，在用户看过的商品中选取购买概率最大的那个商品。那么如何选取特征？重点为us 交互特征，s特征，b特征，ub特征，以及排序特征等等。

In [2]:
action_2_path = "data/JData_Action_201602.csv"
action_3_path = "data/JData_Action_201603.csv"
action_4_path = "data/JData_Action_201604.csv"
comment_path = "data/JData_Comment.csv"
product_path = "data/JData_Product.csv"
user_path = "data/JData_User.csv"

comment_date = ["2016-02-01", "2016-02-08", "2016-02-15", "2016-02-22", "2016-02-29", "2016-03-07", "2016-03-14",
                "2016-03-21", "2016-03-28",
                "2016-04-04", "2016-04-11", "2016-04-15"]

In [3]:
def time_differ_minutes(str1,str2):
    t1=datetime.strptime(str1,'%Y-%m-%d %H:%M:%S')
    t2=datetime.strptime(str2,'%Y-%m-%d %H:%M:%S')
    return ((t1-t2).seconds)/60+1

In [4]:
#计算相差的天数
def diff_of_days1(day1,day2):
    try:
        return (pd.Timestamp(day1)-pd.Timestamp(day2)).days
    except:
        return np.nan

In [5]:
def time_differ_days(str1,str2):
    t1=datetime.strptime(str1,'%Y-%m-%d %H:%M:%S')
    t2=datetime.strptime(str2,'%Y-%m-%d %H:%M:%S')
    return (t1-t2).days

In [6]:
def time_differ_days_nyr(str1,str2):
    t1=datetime.strptime(str1,'%Y-%m-%d')
    t2=datetime.strptime(str2,'%Y-%m-%d')
    return (t1-t2).days

In [7]:
# 相差的分钟数
def diff_of_minutes(day1, day2):
    d = {'1': 0, '2': 31, '3': 60, '4': 91}
    try:
        days = (d[day1[6]] + int(day1[8:10])) - (d[day2[6]] + int(day2[8:10]))
        minutes = int(day2[11:13]) * 60 + int(day2[14:16])
        return (days * 1440 - minutes)
    except:
        return np.nan

In [8]:
# 相差的小时数
def diff_of_hours(day1, day2):
    d = {'1': 0, '2': 31, '3': 60, '4': 91}
    try:
        days = (d[day1[6]] + int(day1[8:10])) - (d[day2[6]] + int(day2[8:10]))
        hours = int(day2[11:13])
        return (days * 24 - hours)
    except:
        return np.nan

In [9]:
def get_action2():
    action2=pd.read_csv(action_2_path)
    return action2

In [10]:
def get_action3():
    action3=pd.read_csv(action_3_path)
    return action3

In [11]:
def get_action4():
    action4=pd.read_csv(action_4_path)
    return action4

In [12]:
def get_actions_cate8(start_date,end_date):
    action2=get_action2()
    action3=get_action3()
    action4=get_action4()
    action=pd.concat([action2,action3,action4],axis=0)
    result=action[(action.time>=start_date)&(action.time<end_date)&(action.cate==8)]
    return result

In [13]:
def get_basic_product_feat():
    dump_path = r'cache/us/product_feat.pkl'
    if os.path.exists(dump_path):
        product = pickle.load(open(dump_path, 'rb+'))
    else:
        product = pd.read_csv(product_path)
        attr1_df = pd.get_dummies(product["a1"], prefix="a1")
        attr2_df = pd.get_dummies(product["a2"], prefix="a2")
        attr3_df = pd.get_dummies(product["a3"], prefix="a3")
        product = pd.concat([product[['sku_id', 'cate', 'brand']], attr1_df, attr2_df, attr3_df], axis=1)
        pickle.dump(product, open(dump_path, 'wb+'))
    return product

In [14]:
#商品comment信息（评论数，差评率）
def get_comments(end_date):
    dump_path = r'cache/us/comments_feat_%s.pkl'% (end_date[:10])
    if os.path.exists(dump_path):
        comments = pickle.load(open(dump_path, 'rb+'))
    else:
        comments = pd.read_csv(comment_path)
        comment_date_end = end_date
        comment_date_begin = comment_date[0]
        for date in reversed(comment_date):
            if date < comment_date_end:
                comment_date_begin = date
                break
        comments = comments[comments.dt == comment_date_begin]
        del comments['dt']
        pickle.dump(comments, open(dump_path, 'wb+'))
    return comments

In [15]:
#数据集 features公共的 特征 如对6种行为的统计 如 user_id,brand,sku_id 对cate8的行为
def get_public_single_time_features(key,start_date,end_date,d):
    if key=='user_id':
        tt='user'
    elif key=='sku_id':
        tt='sku'
    else:
        tt='brand'
    dump_path = r'cache/us/%s_action_time_features_%s_%s.pkl' % (tt,start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        features = pickle.load(open(dump_path, 'rb+'))
    else:
        dataset=get_actions_cate8(start_date,end_date)
        #6种行为统计+时间
        features=None
        if key=='user_id': 
            t=dataset[[key]]
            t.drop_duplicates(inplace=True) #去重
            #6种行为 type=4 用处非常小
            for i in (1,2,3,5,6):
                action=dataset[dataset.type==i][[key]]
                action['%d_%s_type_%s_num'%(d,key[:1],str(i))]=1
                action=action.groupby([key]).agg('sum').reset_index()
                if features is None:
                    features=pd.merge(t,action,on=[key],how='left')
                else:
                    features=pd.merge(features,action,on=[key],how='left')
        elif key=='sku_id':
            # 商品统计行为数
            t=dataset[[key,'brand','cate']]
            t.drop_duplicates(inplace=True) #去重
            for i in (1,2,3,4,5,6):
                action=dataset[dataset.type==i][[key]]
                action['%d_%s_type_%s_num'%(d,key[:1],str(i))]=1
                action=action.groupby([key]).agg('sum').reset_index()
                if features is None:
                    features=pd.merge(t,action,on=[key],how='left')
                else:
                    features=pd.merge(features,action,on=[key],how='left')
            features=features.fillna(0)
            if d==60:
                for i in (1,2,3,4,5,6):
                    #商品在同一品牌中的排序
                    features['%d_%s_type_%s_num_b_rank'%(d,key[:1],str(i))]=features['%d_%s_type_%s_num'%(d,key[:1],str(i))].groupby(features['brand']).rank(ascending=False,method='min')
                    #商品在cate8中的排序
                    features['%d_%s_type_%s_num_c_rank'%(d,key[:1],str(i))]=features['%d_%s_type_%s_num'%(d,key[:1],str(i))].groupby(features['cate']).rank(ascending=False,method='min')
            del features['cate']
            del features['brand']
        else:
            #品牌统计行为数
            t=dataset[[key,'cate']]
            t.drop_duplicates(inplace=True) #去重
            for i in (1,2,3,4,5,6):
                action=dataset[dataset.type==i][[key]]
                action['%d_%s_type_%s_num'%(d,key[:1],str(i))]=1
                action=action.groupby([key]).agg('sum').reset_index()
                if features is None:
                    features=pd.merge(t,action,on=[key],how='left')
                else:
                    features=pd.merge(features,action,on=[key],how='left')
            features=features.fillna(0)
            if d==60:
                for i in (1,2,3,4,5,6):
                    #品牌在cate8中的排序
                    features['%d_%s_type_%s_num_c_rank'%(d,key[:1],str(i))]=features['%d_%s_type_%s_num'%(d,key[:1],str(i))].groupby(features['cate']).rank(ascending=False,method='min')
            del features['cate']
        pickle.dump(features, open(dump_path, 'wb+'))
    return features

In [16]:
#数据集 features公共的 特征 如对6种行为的统计 如 user_id_brand,
def get_public_double_time_features(key1,key2,start_date,end_date,d):
    if key2=='sku_id':
        tt='sku'
    else:
        tt='brand'
    dump_path = r'cache/us/%s_%s_action_time_features_%s_%s.pkl' % (key1[:4],tt,start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        features = pickle.load(open(dump_path, 'rb+'))
    else:
        #u-s or u-b
        #不统计 type=4 没有意义
        dataset=get_actions_cate8(start_date,end_date)
        features=None
        #6种行为统计+时间
        if key2=='sku_id':
            t=dataset[[key1,key2,'brand','cate']]
            t.drop_duplicates(inplace=True) #去重
            #6种行为
            for i in (1,2,3,5,6):
                action=dataset[dataset.type==i][[key1,key2]]
                action['%d_%s%s_type_%s_num'%(d,key1[:1],key2[:1],str(i))]=1
                action=action.groupby([key1,key2]).agg('sum').reset_index()
                if features is None:
                    features=pd.merge(t,action,on=[key1,key2],how='left')
                else:
                    features=pd.merge(features,action,on=[key1,key2],how='left')
            features=features.fillna(0)
            if d==15:
                #浏览次数，加购次数,删购物车次数，收藏，点击
                #一件商品只能收藏一次，故没有排序必要
                for i in (1,6):
                    #us在ud中的排序
                    features['%d_%s%s_type_%s_num_b_rank'%(d,key1[:1],key2[:1],str(i))]=features.groupby(['user_id','brand'])['%d_%s%s_type_%s_num'%(d,key1[:1],key2[:1],str(i))].rank(ascending=False,method='min')
                    #us在uc中的排序
                    features['%d_%s%s_type_%s_num_c_rank'%(d,key1[:1],key2[:1],str(i))]=features.groupby(['user_id','cate'])['%d_%s%s_type_%s_num'%(d,key1[:1],key2[:1],str(i))].rank(ascending=False,method='min')
            del features['brand']
            del features['cate']
        if key2=='brand':
            t=dataset[[key1,key2,'cate']]
            t.drop_duplicates(inplace=True) #去重
            #6种行为
            for i in (1,2,3,5,6):
                action=dataset[dataset.type==i][[key1,key2]]
                action['%d_%s%s_type_%s_num'%(d,key1[:1],key2[:1],str(i))]=1
                action=action.groupby([key1,key2]).agg('sum').reset_index()
                if features is None:
                    features=pd.merge(t,action,on=[key1,key2],how='left')
                else:
                    features=pd.merge(features,action,on=[key1,key2],how='left')
            features=features.fillna(0)
            if d==15:
                #浏览次数，加购次数,删购物车次数，收藏，点击
                for i in (1,2,3,5,6):
                    features['%d_%s%s_type_%s_num_c_rank'%(d,key1[:1],key2[:1],str(i))]=features.groupby(['user_id','cate'])['%d_%s%s_type_%s_num'%(d,key1[:1],key2[:1],str(i))].rank(ascending=False,method='min')
            del features['cate']
        pickle.dump(features, open(dump_path, 'wb+'))
    return features

In [17]:
#获取商品独特的信息 如 交互的人数 
def get_product_unique_time_features(start_date,end_date,d):
    dump_path = r'cache/us/product_unique_time_features_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        features = pickle.load(open(dump_path, 'rb+'))
    else:
        dataset=get_actions_cate8(start_date,end_date)
        features=None
        #6种行为统计+时间
        t=dataset[['sku_id','brand','cate']]
        t.drop_duplicates(inplace=True) #去重
        #交互人数
        tp=dataset[['sku_id','user_id','type']]
        tp.drop_duplicates(inplace=True)
        for i in (1,2,3,4,5,6):
            tt=tp[tp.type==i][['sku_id']]
            tt['%d_s_type_%s_u_num'%(d,str(i))]=1
            tt=tt.groupby(['sku_id']).agg('sum').reset_index()
            if features is None:
                features=pd.merge(t,tt,on='sku_id',how='left')
            else:
                features=pd.merge(features,tt,on='sku_id',how='left')
        features=features.fillna(0)
        if d==60:
            for i in (1,2,3,4,5,6):
                features['%d_s_type_%s_u_num_c_rank'%(d,str(i))]=features['%d_s_type_%s_u_num'%(d,str(i))].groupby(features['cate']).rank(ascending=False,method='min')
                features['%d_s_type_%s_u_num_b_rank'%(d,str(i))]=features['%d_s_type_%s_u_num'%(d,str(i))].groupby(features['brand']).rank(ascending=False,method='min')
        del features['cate']
        del features['brand']
        pickle.dump(features, open(dump_path, 'wb+'))
    return features

In [18]:
#获取商品全局特征信息 如 
def get_product_whole_features(start_date,end_date):
    dump_path = r'cache/us/product_whole_features_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path, 'rb+'))
    else:
        feature = ['sku_id', 'product_1_convern', 'product_2_convern', 'product_3_convern',
                   'product_5_convern', 'product_6_convern', 'product_action_convern']
        actions = get_actions_cate8(start_date, end_date)
        #最后一次交互/购买时间与考察日的差值
        tt=actions[['sku_id','time']]
        #最早交互天数
        t1=tt.groupby('sku_id').agg('min').reset_index()
        t1.rename(columns={'time':'sku_max_action_time'},inplace=True)
        t1['sku_max_action_time']=t1.sku_max_action_time.astype('str').apply(lambda x:time_differ_days(end_date,x))
        #最晚交互天数
        t2=tt.groupby('sku_id').agg('max').reset_index()
        t2.rename(columns={'time':'sku_min_action_time'},inplace=True)
        t2['sku_min_action_time']=t2.sku_min_action_time.astype('str').apply(lambda x:time_differ_days(end_date,x))

        #最早一次交互/购买时间与考察日的差值
        #购买日期
        ts=actions[actions.type==4][['sku_id','time']]
        t3=ts.groupby('sku_id').agg('max').reset_index()
        t3.rename(columns={'time':'sku_min_buy_time'},inplace=True)

        t3['sku_min_buy_time']=t3.sku_min_buy_time.astype('str').apply(lambda x:time_differ_days(end_date,x))

        t4=ts.groupby('sku_id').agg('min').reset_index()
        t4.rename(columns={'time':'sku_max_buy_time'},inplace=True)
        t4['sku_max_buy_time']=t4.sku_max_buy_time.astype('str').apply(lambda x:time_differ_days(end_date,x))

        df = pd.get_dummies(actions['type'], prefix='action')
        actions = pd.concat([actions['sku_id'], df], axis=1)
        actions = actions.groupby(['sku_id'], as_index=False).sum()
        actions['product_1_convern'] = actions['action_4'] / actions['action_1']
        actions['product_2_convern'] = actions['action_4'] / actions['action_2']
        actions['product_3_convern'] = actions['action_4'] / actions['action_3']
        actions['product_5_convern'] = actions['action_4'] / actions['action_5']
        actions['product_6_convern'] = actions['action_4'] / actions['action_6']
        actions['product_action_convern'] = actions[feature[1:6]].sum(axis = 1)
        actions = actions[feature]
        actions=pd.merge(actions,t1,on='sku_id',how='left')
        actions=pd.merge(actions,t2,on='sku_id',how='left')
        actions=pd.merge(actions,t3,on='sku_id',how='left')
        actions=pd.merge(actions,t4,on='sku_id',how='left')
        actions['sku_min_action_time']=actions['sku_min_action_time'].replace(np.nan,-1)
        actions['sku_max_action_time']=actions['sku_max_action_time'].replace(np.nan,-1)
        actions['sku_max_buy_time']=actions['sku_max_buy_time'].replace(np.nan,-1)
        actions['sku_min_buy_time']=actions['sku_min_buy_time'].replace(np.nan,-1)
        actions=actions.fillna(0)
        #只有近15天的交互信息判定为新产品
        actions['new_sku_yes_no']=actions.sku_max_action_time.apply(lambda x:0 if x>=15 else 1)
        pickle.dump(actions, open(dump_path, 'wb+'))
    return actions

In [19]:
#user-product 交互特征
def get_user_product_time_features(start_date,end_date,d):
    dump_path = r'cache/us/user_product_time_features_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        features = pickle.load(open(dump_path, 'rb+'))
    else:
        features=get_actions_cate8(start_date,end_date)
        t=features[['user_id','sku_id','brand','cate']]
        t.drop_duplicates(inplace=True) #去重

        #交互天数
        t1=features[['user_id','sku_id','time']]
        t1['date'] = t1['time'].apply(lambda x:x[:10])
        t1 = t1[['user_id','sku_id','date']].drop_duplicates()
        t1 = t1.groupby(['user_id','sku_id'],as_index=False)['date'].count()
        t1.rename(columns = {'date': '%d_us_active_days' % d},inplace=True)

        t2=features[['user_id','sku_id','time']]
        #交互分钟数
        t2['minute'] = t2['time'].apply(lambda x:x[:16])
        t2 = t2[['user_id','sku_id','minute']].drop_duplicates()
        t2 = t2.groupby(['user_id','sku_id'],as_index=False)['minute'].count()
        t2.rename(columns = {'minute': '%d_us_active_minutes' % d},inplace=True)

        #交互时间
        t4=features[['user_id','sku_id','time']]
        t5 = t4.sort_values(['user_id','sku_id', 'time']) # 先按照uid和time排一下序
        t5['time']=pd.to_datetime(t5['time'])
        t5['pre'] = t5['time'].shift(1)
        t5['uid0'] = t5['user_id'].shift(1)
        t5['sid0'] = t5['sku_id'].shift(1)
        t5['minutes_time'] = (t5['time'] - t5['pre']).map(lambda x:x/np.timedelta64(1, 'm'))
        t6 = t5[(t5['user_id'] == t5['uid0'])&(t5['sku_id'] == t5['sid0'])]
        t6=t6[['user_id','sku_id','minutes_time']]
        t7=t6[t6.minutes_time<5.0][['user_id','sku_id','minutes_time']]
        t7=t7.groupby(['user_id','sku_id']).agg('sum').reset_index()
        t7.rename(columns = {'minutes_time': '%d_us_minutes_time' % d},inplace=True)
        #合并
        features=pd.merge(t,t1,on=['user_id','sku_id'],how='left')
        features=pd.merge(features,t2,on=['user_id','sku_id'],how='left')
        features=pd.merge(features,t7,on=['user_id','sku_id'],how='left')
        features=features.replace(np.nan,0)
        #交互时间排序特征
        if d==60:
            features['%d_us_active_days_rank' % d]=features['%d_us_active_days' % d].groupby(features['user_id']).rank(ascending=False,method='min')
            features['%d_us_active_minutes_rank' % d]=features['%d_us_active_minutes' % d].groupby(features['user_id']).rank(ascending=False,method='min')
            features['%d_us_minutes_time_rank' % d]=features['%d_us_minutes_time' % d].groupby(features['user_id']).rank(ascending=False,method='min')
        del features['brand']
        del features['cate']
        pickle.dump(features, open(dump_path, 'wb+'))
    return features

In [20]:
def get_user_product_whole_features(start_date,end_date):
    dump_path = r'cache/us/user_product_whole_features_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        features = pickle.load(open(dump_path, 'rb+'))
    else:
        #获取 用户商品交互时间的最早 最完时间
         #用户商品交互时间
        user_product=get_actions_cate8(start_date,end_date)
        t=user_product[['user_id','sku_id']]
        t.drop_duplicates(inplace=True) #去重
        t2=user_product[['user_id','sku_id','time']]
        #获取最晚时间，获取最早时间，求差值
        t3=t2.groupby(['user_id','sku_id']).agg('max').reset_index()
        t3['time']=t3.time.astype('str').apply(lambda x:time_differ_days(end_date,x))
        t3.rename(columns={'time':'u_s_max_days'},inplace=True)

        t4=t2.groupby(['user_id','sku_id']).agg('min').reset_index()
        t4['time']=t4.time.astype('str').apply(lambda x:time_differ_days(end_date,x))
        t4.rename(columns={'time':'u_s_min_days'},inplace=True)

        t5=t2.groupby(['user_id','sku_id']).agg('max').reset_index()
        t5['time']=t5.time.astype('str').apply(lambda x:diff_of_hours(end_date,x))
        t5.rename(columns={'time':'u_s_min_hours'},inplace=True)

        t6=t2.groupby(['user_id','sku_id']).agg('min').reset_index()
        t6['time']=t6.time.astype('str').apply(lambda x:diff_of_hours(end_date,x))
        t6.rename(columns={'time':'u_s_max_hours'},inplace=True)
        
        tt=user_product[['user_id','time']]
        t7=user_product[['user_id','sku_id','time']]
        t7.drop_duplicates(inplace=True)
        #user_id max_time
        t8=tt.groupby(['user_id']).agg('max').reset_index()
        t8=pd.merge(t8,t7,on=['user_id','time'],how='left')
        t8['u_last_see_sku']=1
        del t8['time']
        
        t9=tt.groupby(['user_id']).agg('min').reset_index()
        t9=pd.merge(t9,t7,on=['user_id','time'],how='left')
        t9['u_first_see_sku']=1
        del t9['time']
        
        features=pd.merge(t,t3,on=['user_id','sku_id'],how='left')
        features=pd.merge(features,t4,on=['user_id','sku_id'],how='left')
        features=pd.merge(features,t5,on=['user_id','sku_id'],how='left')
        features=pd.merge(features,t6,on=['user_id','sku_id'],how='left')
        features=features.fillna(-1)
        features=pd.merge(features,t8,on=['user_id','sku_id'],how='left')
        features=pd.merge(features,t9,on=['user_id','sku_id'],how='left')
        features=features.fillna(0)
        pickle.dump(features, open(dump_path, 'wb+'))
    return features

In [22]:
#user-brand 交互特征
def get_user_brand_time_features(start_date,end_date,d):
    dump_path = r'cache/us/user_brand_time_features_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        features = pickle.load(open(dump_path, 'rb+'))
    else:
        features=get_actions_cate8(start_date,end_date)
        t=features[['user_id','brand']]
        t.drop_duplicates(inplace=True) #去重

        #交互天数
        t1=features[['user_id','brand','time']]
        t1['date'] = t1['time'].apply(lambda x:x[:10])
        t1 = t1[['user_id','brand','date']].drop_duplicates()
        t1 = t1.groupby(['user_id','brand'],as_index=False)['date'].count()
        t1.rename(columns = {'date': '%d_ub_active_days' % d},inplace=True)

        t2=features[['user_id','brand','time']]
        #交互分钟数
        t2['minute'] = t2['time'].apply(lambda x:x[:16])
        t2 = t2[['user_id','brand','minute']].drop_duplicates()
        t2 = t2.groupby(['user_id','brand'],as_index=False)['minute'].count()
        t2.rename(columns = {'minute': '%d_ub_active_minutes' % d},inplace=True)

        #交互时间
        t4=features[['user_id','brand','time']]
        t5 = t4.sort_values(['user_id','brand', 'time']) # 先按照uid和time排一下序
        t5['time']=pd.to_datetime(t5['time'])
        t5['pre'] = t5['time'].shift(1)
        t5['uid0'] = t5['user_id'].shift(1)
        t5['sid0'] = t5['brand'].shift(1)
        t5['minutes_time'] = (t5['time'] - t5['pre']).map(lambda x:x/np.timedelta64(1, 'm'))
        t6 = t5[(t5['user_id'] == t5['uid0'])&(t5['brand'] == t5['sid0'])]
        t6=t6[['user_id','brand','minutes_time']]
        t7=t6[t6.minutes_time<5.0][['user_id','brand','minutes_time']]
        t7=t7.groupby(['user_id','brand']).agg('sum').reset_index()
        t7.rename(columns = {'minutes_time': '%d_ub_minutes_time' % d},inplace=True)

        #合并
        features=pd.merge(t,t1,on=['user_id','brand'],how='left')
        features=pd.merge(features,t2,on=['user_id','brand'],how='left')
        features=pd.merge(features,t7,on=['user_id','brand'],how='left')
        features=features.replace(np.nan,0)

        #交互时间排序特征
        if d==15:
            features['%d_ub_active_days_rank' % d]=features['%d_ub_active_days' % d].groupby(features['user_id']).rank(ascending=False,method='min')
            features['%d_ub_active_minutes_rank' % d]=features['%d_ub_active_minutes' % d].groupby(features['user_id']).rank(ascending=False,method='min')
            features['%d_ub_minutes_time_rank' % d]=features['%d_ub_minutes_time' % d].groupby(features['user_id']).rank(ascending=False,method='min')
        #加权特征 相对来说 加购物车
        pickle.dump(features, open(dump_path, 'wb+'))
    return features

In [23]:
#获取商品全局特征信息 如 
def get_brand_whole_features(start_date,end_date):
    dump_path = r'cache/us/brand_whole_features_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path, 'rb+'))
    else:
        feature = ['brand', 'brand_1_convern', 'brand_2_convern', 'brand_3_convern',
                   'brand_5_convern', 'brand_6_convern', 'brand_action_convern']
        actions = get_actions_cate8(start_date, end_date)
        df = pd.get_dummies(actions['type'], prefix='action')
        actions = pd.concat([actions['brand'], df], axis=1)
        actions = actions.groupby(['brand'], as_index=False).sum()
        actions['brand_1_convern'] = actions['action_4'] / actions['action_1']
        actions['brand_2_convern'] = actions['action_4'] / actions['action_2']
        actions['brand_3_convern'] = actions['action_4'] / actions['action_3']
        actions['brand_5_convern'] = actions['action_4'] / actions['action_5']
        actions['brand_6_convern'] = actions['action_4'] / actions['action_6']
        actions['brand_action_convern'] = actions[feature[1:6]].sum(axis = 1)
        actions = actions[feature]
        actions=actions.fillna(0)
        actions=actions.replace(np.inf,0)
        pickle.dump(actions, open(dump_path, 'wb+'))
    return actions

In [24]:
def get_user_whole_features(start_date,end_date):
    dump_path = r'cache/us/user_whole_features_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        features = pickle.load(open(dump_path, 'rb+'))
    else:
        dataset=get_actions_cate8(start_date,end_date)
        t=dataset[['user_id']]
        t.drop_duplicates(inplace=True) #去重
         #是否购买过category8商品
        ts=dataset[dataset.type==4][['user_id']]
        ts['user_buy_cate8_yes_no']=1
        ts.drop_duplicates(inplace=True)
        features=pd.merge(t,ts,on='user_id',how='left')
        features['user_buy_cate8_yes_no']=features['user_buy_cate8_yes_no'].replace(np.nan,0)
        pickle.dump(features, open(dump_path, 'wb+'))
    return features

In [25]:
#获取正样本
def get_us_labels(start_date, end_date):
    dump_path = r'cache/us/user_sku_labels_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path, 'rb+'))
    else:
        actions = get_actions_cate8(start_date, end_date)
        actions = actions[actions['type'] == 4]
        actions = actions[['user_id', 'sku_id']].drop_duplicates()
        actions['us_label'] = 1
        pickle.dump(actions, open(dump_path, 'wb+'))
    return actions

In [26]:
def get_u_labels(start_date, end_date):
    dump_path = r'cache/us/user_labels_%s_%s.pkl' % (start_date[:10], end_date[:10])
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path, 'rb+'))
    else:
        actions = get_actions_cate8(start_date, end_date)
        actions = actions[actions['type'] == 4]
        actions = actions[['user_id']].drop_duplicates()
        actions['u_label'] = 1
        pickle.dump(actions, open(dump_path, 'wb+'))
    return actions

In [27]:
#train_end_data 如11-15day 的特征为 10号之前
def make_train_set(train_end_date,test_start_date,test_end_date):
    start_date = "2016-01-31"                                                   #总时间                     
    product = get_basic_product_feat()                                          # 商品基本信息
    comment = get_comments(train_end_date)                                      # 商品评论信息 
    user_whole_infor=get_user_whole_features(start_date,train_end_date)
    brand_whole_infor=get_brand_whole_features(start_date,train_end_date)
    product_whole_infor=get_product_whole_features(start_date,train_end_date)
    user_product_whole_infor=get_user_product_whole_features(start_date,train_end_date)
    us_labels = get_us_labels(test_start_date, test_end_date)                             # 获取label
    u_labels=get_u_labels(test_start_date, test_end_date)  
    actions = None                                                              # 用户商品交互行为次数特征
    user_product_action=None                                                    
    product_action=None
    product_unique_action=None
    user_brand_action=None
    user_brand=None
    #user_all_data=None
    for i in (1, 2, 3, 5, 7, 10, 15,30,60):
        start_days=datetime.strptime(train_end_date,'%Y-%m-%d %H:%M:%S')-timedelta(days=i)
        start_days=start_days.strftime('%Y-%m-%d %H:%M:%S')
        if actions is None:
            #user_notcate8_action=get_user_other_data_time_features(start_days, train_end_date, i)
            #user_action = get_public_single_time_features('user_id',start_days, train_end_date, i)#6个
            #user_unique_action=get_user_unique_time_features(start_days, train_end_date, i)#14
            actions = get_public_double_time_features('user_id','sku_id',start_days,train_end_date,i)#6种行为统计
            user_product_action = get_user_product_time_features(start_days,train_end_date,i)#2个交互时间，交互天数
            
            product_action=get_public_single_time_features('sku_id',start_days, train_end_date, i)
            product_unique_action=get_product_unique_time_features(start_days, train_end_date, i)
            user_brand=get_public_double_time_features('user_id','brand',start_days,train_end_date,i)#6种行为统计
            user_brand_action = get_user_brand_time_features(start_days,train_end_date,i)#2个交互时间，交互天数
        else:
            if (i<61):
                actions = pd.merge(get_public_double_time_features('user_id','sku_id',start_days,train_end_date,i), actions, how='left',
                                    on=['user_id', 'sku_id'])
                user_product_action = pd.merge(get_user_product_time_features(start_days,train_end_date,i), user_product_action, how='left',
                                    on=['user_id', 'sku_id'])
                product_action=pd.merge(get_public_single_time_features('sku_id',start_days, train_end_date, i),product_action,how='left',
                                    on=['sku_id'])
                product_unique_action=pd.merge(get_product_unique_time_features(start_days, train_end_date, i),product_unique_action,how='left',
                                    on=['sku_id'])
                if i==30:
                    user_brand= pd.merge(get_public_double_time_features('user_id','brand',start_days,train_end_date,i), user_brand, how='left',
                                       on=['user_id', 'brand'])
                    user_brand_action= pd.merge(get_user_brand_time_features(start_days,train_end_date,i), user_brand_action, how='left',
                                        on=['user_id', 'brand'])
            else:
                actions = pd.merge(actions,get_public_double_time_features('user_id','sku_id',start_days,train_end_date,i), how='left',
                                    on=['user_id', 'sku_id'])
                user_product_action = pd.merge(user_product_action,get_user_product_time_features(start_days,train_end_date,i), how='left',
                                    on=['user_id', 'sku_id'])
                product_action=pd.merge(product_action,get_public_single_time_features('sku_id',start_days, train_end_date, i),how='left',
                                    on=['sku_id'])
                product_unique_action=pd.merge(product_unique_action,get_product_unique_time_features(start_days, train_end_date, i),how='left',
                                    on=['sku_id'])
                user_brand= pd.merge(user_brand,get_public_double_time_features('user_id','brand',start_days,train_end_date,i),how='left',
                                    on=['user_id', 'brand'])
                user_brand_action= pd.merge(user_brand_action,get_user_brand_time_features(start_days,train_end_date,i),  how='left',
                                    on=['user_id', 'brand'])
                
    #actions = pd.merge(actions, user,                     how='left', on='user_id')
    actions = pd.merge(actions, user_whole_infor,          how='left', on='user_id')
    #actions = pd.merge(actions, user_whole_notcate8_infor,how='left', on='user_id')
    #actions = pd.merge(actions, user_whole_all_infor,     how='left', on='user_id')
    #actions = pd.merge(actions, user_unique_action,       how='left', on='user_id')
    #actions = pd.merge(actions, user_notcate8_action,     how='left', on='user_id')
    #actions = pd.merge(actions, user_all_data,            how='left', on='user_id')#比例特征
    
    actions = pd.merge(actions, product,                  how='left', on='sku_id')
    actions = pd.merge(actions, comment,                  how='left', on='sku_id')
    actions = pd.merge(actions, product_whole_infor,      how='left', on='sku_id')
    
    actions = pd.merge(actions, product_action,           how='left', on='sku_id')
    actions = pd.merge(actions, product_unique_action,    how='left', on='sku_id')
    
    actions = pd.merge(actions, user_product_whole_infor, how='left', on=['user_id', 'sku_id'])
    actions = pd.merge(actions, user_product_action,      how='left', on=['user_id', 'sku_id']) 
    
    actions = pd.merge(actions, brand_whole_infor,        how='left', on=['brand'])
    actions = pd.merge(actions, user_brand,               how='left', on=['user_id','brand'])
    actions = pd.merge(actions, user_brand_action,        how='left', on=['user_id','brand'])
    
    actions = pd.merge(actions, us_labels,                   how='left', on=['user_id', 'sku_id'])
    actions = pd.merge(actions, u_labels,                    how='left', on=['user_id'])
    actions['us_label'] = actions['us_label'].fillna(0)
    actions['u_label'] = actions['u_label'].fillna(0)
    actions = actions.fillna(0)
    return actions, us_labels

In [28]:
def get_train_test():
    
    data1, y_true1 = make_train_set('2016-04-11 00:00:00', '2016-04-11 00:00:00', '2016-04-16 00:00:00')
    data1.to_csv('features/us/data1_features.csv',index=False,encoding='utf-8')
    y_true1.to_csv('features/us/true1_features.csv',index=False,encoding='utf-8')
    del data1
    del y_true1
    gc.collect()
    print '1'
    data0, y_true0 = make_train_set('2016-04-16 00:00:00', '2016-04-16 00:00:00', '2016-04-21 00:00:00')
    data0.to_csv('features/us/data0_features.csv',index=False,encoding='utf-8')
    y_true0.to_csv('features/us/true0_features.csv',index=False,encoding='utf-8')
    del data0
    del y_true0
    gc.collect()
    print '0'

In [29]:
get_train_test()

1


/home/admain/anaconda2/lib/python2.7/site-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)
/home/admain/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/admain/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

0
